In [1]:
import os
import IPython.display as ipd
import numpy as np
import pandas as pd
import warnings
import PIL
from glob import glob
import sklearn as skl
from sklearn.model_selection import train_test_split
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, LabelBinarizer, StandardScaler
import shutil

In [2]:
tracks = utils.load('data/fma_metadata/tracks.csv')
medium = tracks['set', 'subset'] <= 'medium'
y = tracks.loc[medium , ('track', 'genre_top')].to_frame().to_numpy()

In [3]:
y

array([['Hip-Hop'],
       ['Hip-Hop'],
       ['Hip-Hop'],
       ...,
       ['Folk'],
       ['Experimental'],
       ['Rock']], dtype=object)

In [24]:
# getting corrupted files index
warnings.filterwarnings("ignore")
base_path = "data/fma_medium/"
files = sorted(list(glob(base_path + "/*/*.mp3")))
base_out = "grayImages2/"
corruptedRows = []
print(len(files))
for i in range(len(files)):
    file = files[i]
    filename = os.path.basename(file)[:-4]
    path = base_out + filename + ".png"
    if(not os.path.isfile(path)):
        print(i)
        print(path)
        corruptedRows.append(i)

corruptedRows

25000
316
grayImages2/001486.png
977
grayImages2/005574.png
10675
grayImages2/065753.png
13146
grayImages2/080391.png
15626
grayImages2/098558.png
15627
grayImages2/098559.png
15628
grayImages2/098560.png
15634
grayImages2/098571.png
15836
grayImages2/099134.png
16305
grayImages2/105247.png
16959
grayImages2/108925.png
20621
grayImages2/126981.png
20780
grayImages2/127336.png
21988
grayImages2/133297.png
23620
grayImages2/143992.png


[316,
 977,
 10675,
 13146,
 15626,
 15627,
 15628,
 15634,
 15836,
 16305,
 16959,
 20621,
 20780,
 21988,
 23620]

In [20]:
Labels = np.unique(y)
Labels[11] = "Old-Time" # renaming
print(Labels)

label_encoder = sklearn.preprocessing.LabelEncoder()
onehot_encoder = sklearn.preprocessing.OneHotEncoder(sparse = False)

integer_encoded_y = label_encoder.fit_transform(y.reshape(1, y.shape[0])[0])
integer_encoded_y = integer_encoded_y.reshape(len(integer_encoded_y), 1)
integer_encoded_y = np.delete(integer_encoded_y, corruptedRows,axis = 0)

#removing Corrupted Labels
print(integer_encoded_y.shape)
onehot_encoded_y = onehot_encoder.fit_transform(integer_encoded_y)

print(onehot_encoded_y.shape)
print(onehot_encoded_y[0])

['Blues' 'Classical' 'Country' 'Easy Listening' 'Electronic'
 'Experimental' 'Folk' 'Hip-Hop' 'Instrumental' 'International' 'Jazz'
 'Old-Time' 'Pop' 'Rock' 'Soul-RnB' 'Spoken']
(24985, 1)
(24985, 16)
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [33]:
#Moving images to proper Directory
baseImagePath = 'grayImages2'
imagesPaths= sorted(list(glob(baseImagePath + "/*.png")))
imageCount = len(imagesPaths)
for i in range(imageCount):
    label = Labels[integer_encoded_y[i]]
    folder = str(label[0])
    dst = "spectrogramDirectory/" + folder
#     print(dst)
    shutil.move(imagesPaths[i],dst)